In [1]:
from FourierCKKS import *

In [2]:
poly_degree = 1024
message_length = 256

In [3]:
fcks = FourierCKKS(poly_degree=poly_degree)

In [4]:
orig1 = np.random.random(message_length) + 1j * np.random.random(message_length)
orig2 = np.random.random(message_length) + 1j * np.random.random(message_length)

In [5]:
ct1 = fcks.forward(orig1)
ct2 = fcks.forward(orig2)

In [6]:
rec1 = fcks.backward(ct1)
rec2 = fcks.backward(ct2)
success1 = np.allclose(rec1[:message_length], orig1, atol=1e-1, rtol=1e-1)
success2 = np.allclose(rec2[:message_length], orig2, atol=1e-1, rtol=1e-1)
print(f"Recovery1 successful: {success1}, Recovery2 successful: {success2}")

Recovery1 successful: True, Recovery2 successful: True


In [7]:
ct_sum = fcks.cipher_add(ct1, ct2)
rec_sum = fcks.backward(ct_sum)
true_sum = orig1 + orig2
add_success = np.allclose(rec_sum[:message_length], true_sum, atol=1e-1, rtol=1e-1)
print(f"Addition recovery successful: {add_success}")

Addition recovery successful: True


In [8]:
ct_conv = fcks.cipher_conv(ct1, ct2)
rec_conv = fcks.backward(ct_conv)
from fft.fft import linear_convolution_direct
true_conv = linear_convolution_direct(orig1, orig2)
conv_success = np.allclose(rec_conv[:2*message_length - 1], true_conv, atol=1e-1, rtol=1e-1)
print(f"Convolution recovery successful: {conv_success}")

Convolution recovery successful: True
